# Phase2 - RAG

If not already done run this in the top level folder:
```
pip install -r requirements.txt
```




In [ ]:
import json
import requests
import os
import openai
from enum import Enum
from pydantic import BaseModel
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

This is the object model for receiving questions.

In [ ]:

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_false = "true_false"
    popular_choice = "popular_choice"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


## Load custom data
Here's how you load custom data into an embedding model and how you use langchain to query it.


In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.chains import RetrievalQA

import os
from langchain.llms import AzureOpenAI
from langchain_openai import AzureChatOpenAI


# use an embeddingsmodel to create embeddings
embeddings_model = AzureOpenAIEmbeddings(    
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_version = os.getenv("OPENAI_EMBEDDING_API_VERSION"),
    model= os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
)

AZURE_OPENAI_VERSION="2024-02-01"
# Create an instance of Azure OpenAI
llm = AzureChatOpenAI(
    azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
    api_version=AZURE_OPENAI_VERSION
)

# load your data
data_dir = "data/movies"
documents = DirectoryLoader(path=data_dir, glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader).load()

#create chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
document_chunks = text_splitter.split_documents(documents)

# create a qdrant index in memory
qdrant = Qdrant.from_documents(
    document_chunks,
    embeddings_model,
    location=":memory:",
    collection_name="movies",
)

# create a retriever to allow a rag flow
retriever = qdrant.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
loader = DirectoryLoader(path=data_dir, glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)

index = VectorstoreIndexCreator(embedding=embeddings_model).from_loaders([loader])



In [ ]:
query = "Tell me about the latest Ant Man movie. When was it released? What is it about?"
index.query(llm=llm, question=query)

# YOUR Mission: 
Adjust the function below and reuse it in the main.py file later to deploy to Azure and to update your service. 
Ensure the answers provided are correct and in the correct format.



In [ ]:

async def ask_question(ask: Ask):
    """
    Ask a question
    """
    answer = index.query(llm=llm, question=ask.question)
    
    print(answer)
    return answer

Use this snippet to try your method with several questions.

In [ ]:

ask = Ask(question="Tell me about the latest Ant Man movie. When was it released? ", type=QuestionType.estimation)
answer = await ask_question(ask)
print('Answer:', answer)


Make sure you transfer your code changes into main.py (or additional files). Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase2 $AZURE_ENV_NAME
```
